In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('dark_background')
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

# Check the availability of GPU

In [ ]:
torch.cuda.is_available()

True

# Use GPU as a device for Training and Testing

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Mount the drive and set the path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Course Material/Supervised Machine Learning/Scheduling Learning Rate

/content/drive/MyDrive/Course Material/Supervised Machine Learning/Scheduling Learning Rate


# Import the dataset

In [ ]:
import pandas as pd

data = pd.read_csv('mnist_train.csv')

In [ ]:
data.shape

(60000, 785)

In [ ]:
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Transforming the data into Numpy array and Extracting only 10000 samples

In [ ]:
data = np.array(data)
data = data[0:10000,:] # Extracting first 10000 samples

# Extracting Features and Labels

In [ ]:
X = data[:,1:]
y = data[:,0]

In [ ]:
X.shape

(10000, 784)

# Four Important steps to use data in Pytorch

In [ ]:
# Step 1: Convert to pytorch tensor
X_t   = torch.tensor( X, dtype = torch.float32 )
y_t = torch.tensor( y, dtype = torch.long)

# Step 2: Perform Train Test Split and Normalize the data
train_data, test_data, train_labels, test_labels = train_test_split(X_t, y_t, test_size = 0.1)

train_data = train_data/torch.max(train_data)
test_data  = test_data/torch.max(test_data)


# Step 3: Convert into PyTorch Datasets
X_train = TensorDataset(train_data,train_labels)
X_test  = TensorDataset(test_data,test_labels)


# Step 4: Move the data into dataloader objects
batchsize    = 32
train_loader = DataLoader(X_train, batch_size = batchsize, shuffle = True, drop_last=True)
test_loader  = DataLoader(X_test, batch_size = X_test.tensors[0].shape[0])

# Create the Neural Network model

In [ ]:
class NN_Classifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.input = nn.Linear(784,64)
    self.fc1 = nn.Linear(64,32)
    self.fc2 = nn.Linear(32,32)

    # Classification OR  output layer

    self.output = nn.Linear(32,10)

  # Forward pass

  def forward(self,x):

    # Pass the data through the input layer

    x = self.input(x)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)

    # output layer

    x = self.output(x)
    # X = torch.sigmoid
    return x

In [ ]:
model = NN_Classifier()

# Calculating stepsize for scheduling Learning rate

In [ ]:
len(train_loader) # Number of iterations, 9000/32 = 281

281

In [ ]:
stepsize  = 10 * len(train_loader) # equal to 10 epochs
stepsize

2810

# Train and Test the model

In [ ]:
epochs = 50

lossfunc = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), momentum = 0.9, lr = 0.01)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = stepsize , gamma = 0.5)

model.to(device) # placing a model on GPU

# Initialize losses and Accuracies

losses    = torch.zeros(epochs)
trainAcc  = []
testAcc   = []


# For Loop for epochs

for epoch in range(epochs):
 model.train()

 # Loop for training in batches

 batchAcc  = []
 batchLoss = []

 for X,y in train_loader:

  X = X.to(device) # Placing features on device
  y = y.to(device) # placing labels on device

  ypred = model(X)
  loss = lossfunc(ypred,y)

# Backpropagation

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  scheduler.step()

  # Append batchloss

  batchLoss.append(loss.item())

  ypred = ypred.cpu() # Back to cpu
  y = y.cpu()

  # compute batch accuracy

  acc = 100 * torch.mean((torch.argmax(ypred, dim=1) == y).float())
  batchAcc.append(acc)

# Batch loop concluded

# Average training accuracy (Outside the batchloop)

 trainAcc.append(np.mean(batchAcc))

# Losses through epochs

 losses[epoch] = np.mean(batchLoss)

# TESTING the model

 model.eval()
 X,y = next(iter(test_loader))
 X = X.to(device)
 y = y.to(device)

 with torch.no_grad(): #This step  Deactivating autogradient
  ypred = model(X)
  ypred = ypred.cpu()
  y = y.cpu()

 # Appending test accuracy

  testAcc.append(100 * torch.mean((torch.argmax(ypred,dim=1)==y).float()))
  print(f'epoch {epoch}: Learning Rate={scheduler.get_last_lr()[0]}')



epoch 0: Learning Rate=0.01
epoch 1: Learning Rate=0.01
epoch 2: Learning Rate=0.01
epoch 3: Learning Rate=0.01
epoch 4: Learning Rate=0.01
epoch 5: Learning Rate=0.01
epoch 6: Learning Rate=0.01
epoch 7: Learning Rate=0.01
epoch 8: Learning Rate=0.01
epoch 9: Learning Rate=0.005
epoch 10: Learning Rate=0.005
epoch 11: Learning Rate=0.005
epoch 12: Learning Rate=0.005
epoch 13: Learning Rate=0.005
epoch 14: Learning Rate=0.005
epoch 15: Learning Rate=0.005
epoch 16: Learning Rate=0.005
epoch 17: Learning Rate=0.005
epoch 18: Learning Rate=0.005
epoch 19: Learning Rate=0.0025
epoch 20: Learning Rate=0.0025
epoch 21: Learning Rate=0.0025
epoch 22: Learning Rate=0.0025
epoch 23: Learning Rate=0.0025
epoch 24: Learning Rate=0.0025
epoch 25: Learning Rate=0.0025
epoch 26: Learning Rate=0.0025
epoch 27: Learning Rate=0.0025
epoch 28: Learning Rate=0.0025
epoch 29: Learning Rate=0.00125
epoch 30: Learning Rate=0.00125
epoch 31: Learning Rate=0.00125
epoch 32: Learning Rate=0.00125
epoch 33: L